# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth (23.4 Deg. perpendicular to the ecliptic).
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 60 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import csv
import json
from pprint import pprint
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "../output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

612

In [4]:
df = pd.DataFrame({
    "City": cities
})

df.to_csv(output_data_file)

data = pd.read_csv("../output_data/cities.csv")

data["City"] = cities
data["Cloudiness"] = ""
data["Country"] = ""
data["date"] = ""
data["humidity"] = ""
data["lat"] = ""
data["lon"] = ""
data["max temp"] = ""
data["wind speed"] = ""


column_names = ["City", "Country", "Date", "Cloudiness", "Humidity", "Lat", "Lng", "Max Temp", "Wind Speed"]

data = pd.DataFrame(data, columns=column_names)
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '../output_data/cities.csv'

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
units = "imperial"
url = "http://api.openweathermap.org/data/2.5/weather?"

params = {
    "key": api_key,
    "units": units,
}

for index, row in data.iterrows():
    
    time.sleep(1)
    
    city = row['City']
    
    query_url = url + "appid=" + api_key + "&" + units + "&q=" + city
    
    print(f"Retrieving Results for Index {index}: {city}.")
    
    response = requests.get(query_url).json()
    
    try:
        data.loc[index, 'Country'] = response["sys"]["country"]
        data.loc[index, "Date"] = response["dt"]
        data.loc[index, 'Cloudiness'] = str(response["clouds"]["all"])
        data.loc[index, "Humidity"] = response["main"]["humidity"]


        data.loc[index, 'Lat'] = response["coord"]["lat"]
        data.loc[index, 'Lng'] = response["coord"]["lon"]
        data.loc[index, "Max Temp"] = response["main"]["temp_max"]
        data.loc[index, "Wind Speed"] = response["wind"]["speed"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
data = data.dropna(axis = 0, how ='any') 
data.to_csv(output_data_file)
data.head()

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
#Design the scatter plot
city_lat = data["Lat"]
max_temp = 1.8*(data["Max Temp"] - 273) + 32

plt.scatter(city_lat, max_temp,marker="o", facecolors="blue", edgecolors="black", alpha=0.42)

#Create title and labels
plt.title(f"City Latitude vs. Max Temperature (04/15/2019)")

plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")
plt.grid()

#Save an image of the chart
plt.savefig("../Images/lat_max_temp_scatter_plot.png")
plt.show()


#### Latitude vs. Humidity Plot

In [ ]:
#Design the scatter plot
city_lat = data["Lat"]
humid_data = data["Humidity"]


plt.scatter(city_lat, humid_data,marker="o", facecolors="blue", edgecolors="black", alpha=0.42)

#Create title and labels
plt.title("City Latitude vs. Humidity (04/15/2019)")

plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.ylim(0, 105)
plt.grid()

#Save an image of the chart
plt.savefig("../Images/lat_humid_scatter_plot.png")
plt.show()

#### Latitude vs. Cloudiness Plot

In [ ]:
#Design the scatter plot
city_lat = data["Lat"]
cloud_data = data["Cloudiness"]


plt.scatter(city_lat, cloud_data, marker="o", facecolors="blue", edgecolors="black", alpha=0.6)

#Create title and labels
plt.title("City Latitude vs. Cloudiness (04/15/2019)")

plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.grid()

#Save an image of the chart
plt.savefig("../Images/lat_cloud_scatter_plot.png")
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
#Design the scatter plot
city_lat = data["Lat"]
wind_speed_data = data["Wind Speed"]


plt.scatter(city_lat, wind_speed_data,marker="o", facecolors="blue", edgecolors="black", alpha=0.42)

#Create title and labels
plt.title("City Latitude vs. Wind Speed (4/15/2019)")

plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.grid()

#Save an image of the chart
plt.savefig("../Images/lat_wind_speed_scatter_plot.png")
plt.show()